# Credit Risk Scorecard Model Validation

## Before you begin
To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also create a documentation project on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
# %pip install --upgrade validmind

## Initialize the client library
In a browser, go to the Client Integration page of your documentation project and click Copy to clipboard next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

This step requires a documentation project. Learn how you can create one.

Next, replace this placeholder with your own code snippet:

In [2]:


import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "b5f4534a7bde0407aa279896129b0b69",
  api_secret = "0c7cd7fef1692e26518dea6a2e4f93467d37396c214949562480818190220cbb",
  project = "clmotr3oa000aesy6c54wqb6g"
)
  

InvalidProjectError: Invalid project

## Introduction

The **Credit risk Scorecard** model created from the Lending Club dataset is instrumental in computing the Probability of Default (PD), a key factor in ECL calculations. This scorecard assesses several credit characteristics of potential borrowers, like their credit history, income, outstanding debts, and more, each of which is assigned a specific score. By combining these scores, we derive a total score for each borrower, used to estimate the Point-in-Time (PiT) PD.

## Setup

### Import Libraries

In [ ]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

from IPython.display import HTML
from notebooks.probability_of_default.helpers.Developer import Developer
from notebooks.probability_of_default.helpers.scorecard_model import *
from notebooks.probability_of_default.helpers.model_development import *

# Visualization imports
%matplotlib inline

### Input Parameters

In [ ]:
default_column = "default"

### Load Datasets and Models

In [ ]:
developer = Developer()
scorecard = developer.load_objects_from_pickle("datasets/scorecard_data_and_models.pkl")

df_raw = scorecard["df_raw"]
df_prepared = scorecard["df_prepared"]
df_train = scorecard["df_train"]
df_train_feature_selection = scorecard["df_train_feature_selection"]

df_train_feature_eng = scorecard["df_train_feature_eng"]
df_test_feature_eng = scorecard["df_test_feature_eng"]

model_fit_final = scorecard["model_fit_final"]

### Create ValidMind Datasets and Model

In [ ]:
from validmind.vm_models.test_context import TestContext

vm_df_raw = vm.init_dataset(dataset=df_raw, target_column=default_column)
vm_df_prepared = vm.init_dataset(dataset=df_prepared, target_column=default_column)
vm_df_train_feature_selection = vm.init_dataset(dataset=df_train_feature_selection, target_column=default_column)

vm_df_train_feature_eng = vm.init_dataset(dataset=df_train_feature_eng, target_column=default_column)
vm_df_test_feature_eng = vm.init_dataset(dataset=df_test_feature_eng, target_column=default_column)

vm_model_fit_final = vm.init_model(
    model = model_fit_final, 
    train_ds=vm_df_train_feature_eng, 
    test_ds=vm_df_test_feature_eng)


test_context_raw = TestContext(dataset=vm_df_raw)
test_context_prepared = TestContext(dataset=vm_df_prepared)
test_context_train_feature_selection = TestContext(dataset=vm_df_train_feature_selection)

test_context_models_fit_final = TestContext(models = [vm_model_fit_final])
test_context_model_fit_final = TestContext(model = [vm_model_fit_final])

## Model validation

### Data Description

In [ ]:
from validmind.tests.data_validation.DescriptiveStatistics import DescriptiveStatistics

metric = DescriptiveStatistics(test_context_raw, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.MissingValuesBarPlot import MissingValuesBarPlot

params = {"threshold": 80,
          "fig_height": 1100}

metric = MissingValuesBarPlot(test_context_raw, params)
metric.run()
await metric.result.log()
metric.result.show()

### Data Preparation

In [ ]:
from validmind.tests.data_validation.ClassImbalance import ClassImbalance

metric = ClassImbalance(test_context_prepared, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.IQROutliersTable import IQROutliersTable

numerical_features = get_numerical_columns(df_prepared)
params = {"num_features": numerical_features,
          "threshold": 1.5
        }

metric = IQROutliersTable(test_context_prepared, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.IQROutliersBarPlot import IQROutliersBarPlot

numerical_features = get_numerical_columns(df_prepared)
params = {"num_features": numerical_features,
          "threshold": 1.5,
          "fig_width": 500}

metric = IQROutliersBarPlot(test_context_prepared, params)
metric.run()
await metric.result.log()
metric.result.show()

### Exploratory Data Analysis

In [ ]:
from validmind.tests.data_validation.TabularNumericalHistograms import TabularNumericalHistograms

metric = TabularNumericalHistograms(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.HighCardinality import HighCardinality
metric = HighCardinality(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.TabularCategoricalBarPlots import TabularCategoricalBarPlots
metric = TabularCategoricalBarPlots(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.TargetRateBarPlots import TargetRateBarPlots

# Configure the metric
params = {
    "default_column": default_column,
    "columns": None
}

metric = TargetRateBarPlots(test_context_train_feature_selection, params=params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.ChiSquaredFeaturesTable import ChiSquaredFeaturesTable

categorical_features = get_categorical_columns(df_train_feature_selection)
params = {"cat_features": categorical_features,
          "p_threshold": 0.05}

metric = ChiSquaredFeaturesTable(test_context_train_feature_selection, params)
metric.run()
await metric.result.log() 
metric.result.show()

In [ ]:
from validmind.tests.data_validation.ANOVAOneWayTable import ANOVAOneWayTable

numerical_features = get_numerical_columns(df_train_feature_selection)
params = {"num_features": numerical_features,
          "p_threshold": 0.05}

metric = ANOVAOneWayTable(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.PearsonCorrelationMatrix import PearsonCorrelationMatrix

params = {"declutter": False,
          "features": None,
          "fontsize": 13}

metric = PearsonCorrelationMatrix(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.FeatureTargetCorrelationPlot import FeatureTargetCorrelationPlot

params = {"features": None}

metric = FeatureTargetCorrelationPlot(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

### Feature Engineering

In [ ]:
from validmind.tests.data_validation.WOEBinTable import WOEBinTable

metric = WOEBinTable(test_context_train_feature_selection, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

metric = WOEBinTable(test_context_train_feature_selection, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.data_validation.WOEBinPlots import WOEBinPlots

params = {
    "breaks_adj": {"int_rate": [5,10,15]},
    "fig_height": 500,
}

metric = WOEBinPlots(test_context_train_feature_selection, params=params)
metric.run()
await metric.result.log()
metric.result.show()

### Model Training

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionCoeffsPlot import RegressionCoeffsPlot

metric = RegressionCoeffsPlot(test_context_models_fit_final, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.RegressionModelsCoeffs import RegressionModelsCoeffs

metric = RegressionModelsCoeffs(test_context_models_fit_final, params=None)
metric.run()
await metric.result.log()
metric.result.show()

### Model Evaluation

In [ ]:
from validmind.tests.model_validation.statsmodels.GINITable import GINITable

metric = GINITable(test_context_model_fit_final, params=None)
metric.run()
await metric.result.log() 
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.LogRegressionConfusionMatrix import LogRegressionConfusionMatrix

params = {
    "cut_off_threshold": 0.5
}

metric = LogRegressionConfusionMatrix(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.sklearn.ROCCurve import ROCCurve

metric = ROCCurve(test_context_model_fit_final, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.sklearn.PrecisionRecallCurve import PrecisionRecallCurve

metric = PrecisionRecallCurve(test_context_model_fit_final, params=None)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.LogisticRegPredictionHistogram import LogisticRegPredictionHistogram

# Configure test parameters
params = {
    "title": "Histogram of Probability of Default",
}

metric = LogisticRegPredictionHistogram(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.LogisticRegCumulativeProb import LogisticRegCumulativeProb

# Configure test parameters
params = {
    "title": "Cumulative Probability of Default",
}

metric = LogisticRegCumulativeProb(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [ ]:
from validmind.tests.model_validation.statsmodels.ScorecardHistogram import ScorecardHistogram

# Configure test parameters
params = {
    "target_score": 600,
    "target_odds": 50,
    "pdo": 20,
    "title": "Histogram of Credit Scores",
}

metric = ScorecardHistogram(test_context_model_fit_final, params)
metric.run()
await metric.result.log()
metric.result.show()